In [796]:
import get_stock_data
from datetime import date
import sqlite3
import config
import pandas as pd
import numpy as np

# Stocks

## Getting Stock Data

In [797]:


TICKER='BA'
START_DATE='2013-01-01'
END_DATE = date.today().strftime('%Y-%m-%d')
Y_VAR=['Close']
VARS_TO_LAG=['Open', 'High', 'Low', 'Close', 'Volume']
NUM_OF_LAGS=30

In [798]:
df = get_stock_data.get_daily_stock_data(TICKER, START_DATE, END_DATE)
df_original = df.copy()
df




,Open,High,Low,Close,Volume,Date,Ticker
0,64.148095,64.801726,64.064291,64.583847,5036400,2013-01-02,BA
1,64.516799,65.346413,63.997250,64.919037,3987000,2013-01-03,BA
2,65.187206,65.379941,64.893911,65.103409,3431700,2013-01-04,BA
3,64.977719,65.103418,63.134138,63.796150,7172600,2013-01-07,BA
4,63.050332,63.527986,61.265416,62.120163,20268100,2013-01-08,BA
...,...,...,...,...,...,...,...
2818,182.000000,184.940002,180.520004,181.149994,10016200,2024-03-14,BA
2819,180.990005,184.179993,180.910004,182.529999,10708600,2024-03-15,BA
2820,180.259995,181.190002,177.520004,179.839996,9497800,2024-03-18,BA
2821,180.110001,181.860001,179.740005,181.139999,7648000,2024-03-19,BA


## Create - Price Movement Indicator

In [799]:


df[f'{Y_VAR[0]}_change'] = df[f'{Y_VAR[0]}'].diff().apply(lambda x: 1 if x > 0 else 0)

df

,Open,High,Low,Close,Volume,Date,Ticker,Close_change
0,64.148095,64.801726,64.064291,64.583847,5036400,2013-01-02,BA,0
1,64.516799,65.346413,63.997250,64.919037,3987000,2013-01-03,BA,1
2,65.187206,65.379941,64.893911,65.103409,3431700,2013-01-04,BA,1
3,64.977719,65.103418,63.134138,63.796150,7172600,2013-01-07,BA,0
4,63.050332,63.527986,61.265416,62.120163,20268100,2013-01-08,BA,0
...,...,...,...,...,...,...,...,...
2818,182.000000,184.940002,180.520004,181.149994,10016200,2024-03-14,BA,0
2819,180.990005,184.179993,180.910004,182.529999,10708600,2024-03-15,BA,1
2820,180.259995,181.190002,177.520004,179.839996,9497800,2024-03-18,BA,0
2821,180.110001,181.860001,179.740005,181.139999,7648000,2024-03-19,BA,1


## Calculating Returns

In [800]:
for var in VARS_TO_LAG:
    df[f'log_{var}'] = np.log(df[f'{var}'] / df[f'{var}'].shift(1))


## Calculating Lag

In [801]:

for lag in range(1, NUM_OF_LAGS + 1):
    for var in VARS_TO_LAG:
        df[f'log_{var}_lag{lag}'] = df[f'log_{var}'].shift(lag)

df


/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/3547659935.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_{var}_lag{lag}'] = df[f'log_{var}'].shift(lag)
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/3547659935.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'log_{var}_lag{lag}'] = df[f'log_{var}'].shift(lag)
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/3547659935.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of call

,Open,High,Low,Close,Volume,Date,Ticker,Close_change,log_Open,log_High,...,log_Open_lag29,log_High_lag29,log_Low_lag29,log_Close_lag29,log_Volume_lag29,log_Open_lag30,log_High_lag30,log_Low_lag30,log_Close_lag30,log_Volume_lag30
0,64.148095,64.801726,64.064291,64.583847,5036400,2013-01-02,BA,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64.516799,65.346413,63.997250,64.919037,3987000,2013-01-03,BA,1,0.005731,0.008370,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,65.187206,65.379941,64.893911,65.103409,3431700,2013-01-04,BA,1,0.010338,0.000513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64.977719,65.103418,63.134138,63.796150,7172600,2013-01-07,BA,0,-0.003219,-0.004238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,63.050332,63.527986,61.265416,62.120163,20268100,2013-01-08,BA,0,-0.030111,-0.024497,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,182.000000,184.940002,180.520004,181.149994,10016200,2024-03-14,BA,0,-0.006353,-0.005554,...,0.042608,0.004015,0.017364,-0.005845,-0.741183,0.006217,0.046879,0.020429,0.051533,0.520124
2819,180.990005,184.179993,180.910004,182.529999,10708600,2024-03-15,BA,1,-0.005565,-0.004118,...,-0.022607,-0.024001,-0.002227,-0.002052,-0.532330,0.042608,0.004015,0.017364,-0.005845,-0.741183
2820,180.259995,181.190002,177.520004,179.839996,9497800,2024-03-18,BA,0,-0.004042,-0.016367,...,-0.021369,-0.010555,-0.021306,-0.013221,0.325342,-0.022607,-0.024001,-0.002227,-0.002052,-0.532330
2821,180.110001,181.860001,179.740005,181.139999,7648000,2024-03-19,BA,1,-0.000832,0.003691,...,0.006721,0.015791,0.009313,0.009393,-0.138848,-0.021369,-0.010555,-0.021306,-0.013221,0.325342


## Calculate Simple Moving Averages

In [802]:
LONG_WINDOW = 200
MID_WINDOW = 50
SHORT_WINDOW = 10
df['sma_long'] = df['Close'].rolling(window=LONG_WINDOW).mean()
df['sma_mid'] = df['Close'].rolling(window=MID_WINDOW).mean()
df['sma_short'] = df['Close'].rolling(window=SHORT_WINDOW).mean()

/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/234885322.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['sma_long'] = df['Close'].rolling(window=LONG_WINDOW).mean()
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/234885322.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['sma_mid'] = df['Close'].rolling(window=MID_WINDOW).mean()
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/234885322.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the res

## Calc Rolling Min and Rolling Max

In [803]:
df['min'] = df['Close'].rolling(window=MID_WINDOW).min()
df['max'] = df['Close'].rolling(window=MID_WINDOW).max()

/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/2947537239.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['min'] = df['Close'].rolling(window=MID_WINDOW).min()
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/2947537239.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['max'] = df['Close'].rolling(window=MID_WINDOW).max()


## Calc Momentum and Volatility of Returns

In [804]:
df['mom'] = df['log_Close'].rolling(window=MID_WINDOW).mean()
df['vol'] = df['log_Close'].rolling(window=MID_WINDOW).std()

/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/299884824.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['mom'] = df['log_Close'].rolling(window=MID_WINDOW).mean()
/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/299884824.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['vol'] = df['log_Close'].rolling(window=MID_WINDOW).std()


## Dropping Rows with Leakage

In [805]:
df = df.drop(columns=['log_Close','Close', 'Ticker','Open', 'High', 'Low', 'Volume', 'log_Open', 'log_High', 'log_Low',  'log_Volume'])
df

,Date,Close_change,log_Open_lag1,log_High_lag1,log_Low_lag1,log_Close_lag1,log_Volume_lag1,log_Open_lag2,log_High_lag2,log_Low_lag2,...,log_Low_lag30,log_Close_lag30,log_Volume_lag30,sma_long,sma_mid,sma_short,min,max,mom,vol
0,2013-01-02,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-03,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-04,1,0.005731,0.008370,-0.001047,0.005177,-0.233652,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-07,0,0.010338,0.000513,0.013914,0.002836,-0.149983,0.005731,0.008370,-0.001047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-08,0,-0.003219,-0.004238,-0.027492,-0.020284,0.737212,0.010338,0.000513,0.013914,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2818,2024-03-14,0,-0.027358,-0.013937,-0.005914,-0.010311,-0.607309,-0.031222,-0.038949,-0.035611,...,0.020429,0.051533,0.520124,213.98925,208.221801,194.443001,181.149994,249.000000,-0.006583,0.024857
2819,2024-03-15,1,-0.006353,-0.005554,-0.008605,-0.006603,-0.017802,-0.027358,-0.013937,-0.005914,...,0.017364,-0.005845,-0.741183,213.87845,206.994200,192.696001,181.149994,249.000000,-0.005798,0.024668
2820,2024-03-18,0,-0.005565,-0.004118,0.002158,0.007589,0.066843,-0.006353,-0.005554,-0.008605,...,-0.002227,-0.002052,-0.532330,213.74915,205.692200,190.626001,179.839996,249.000000,-0.006179,0.024657
2821,2024-03-19,1,-0.004042,-0.016367,-0.018916,-0.014847,-0.119987,-0.005565,-0.004118,0.002158,...,-0.021306,-0.013221,0.325342,213.61505,204.335000,188.626001,179.839996,229.000000,-0.006364,0.024518


## Dropping Rows with Any NAs

In [806]:
df.dropna(inplace=True)
df = df.sort_values(by='Date', ascending=False)
df

df

,Date,Close_change,log_Open_lag1,log_High_lag1,log_Low_lag1,log_Close_lag1,log_Volume_lag1,log_Open_lag2,log_High_lag2,log_Low_lag2,...,log_Low_lag30,log_Close_lag30,log_Volume_lag30,sma_long,sma_mid,sma_short,min,max,mom,vol
2822,2024-03-20,1,-0.000832,0.003691,0.012428,0.007203,-0.216616,-0.004042,-0.016367,-0.018916,...,0.009313,0.009393,-0.138848,213.487350,203.510600,187.304001,179.839996,227.839996,-0.003969,0.022578
2821,2024-03-19,1,-0.004042,-0.016367,-0.018916,-0.014847,-0.119987,-0.005565,-0.004118,0.002158,...,-0.021306,-0.013221,0.325342,213.615050,204.335000,188.626001,179.839996,229.000000,-0.006364,0.024518
2820,2024-03-18,0,-0.005565,-0.004118,0.002158,0.007589,0.066843,-0.006353,-0.005554,-0.008605,...,-0.002227,-0.002052,-0.532330,213.749150,205.692200,190.626001,179.839996,249.000000,-0.006179,0.024657
2819,2024-03-15,1,-0.006353,-0.005554,-0.008605,-0.006603,-0.017802,-0.027358,-0.013937,-0.005914,...,0.017364,-0.005845,-0.741183,213.878450,206.994200,192.696001,181.149994,249.000000,-0.005798,0.024668
2818,2024-03-14,0,-0.027358,-0.013937,-0.005914,-0.010311,-0.607309,-0.031222,-0.038949,-0.035611,...,0.020429,0.051533,0.520124,213.989250,208.221801,194.443001,181.149994,249.000000,-0.006583,0.024857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2013-10-22,1,0.001550,-0.000977,-0.001568,-0.008607,-0.201546,0.022204,0.003833,0.011947,...,0.007852,0.009101,0.280226,81.528703,95.833602,102.025521,87.481590,104.334114,0.003225,0.013230
202,2013-10-21,0,0.022204,0.003833,0.011947,0.001879,-0.047282,0.008634,0.014236,0.008887,...,0.013701,0.010504,-0.115993,81.326183,95.522949,101.426015,87.481590,104.334114,0.002851,0.013348
201,2013-10-18,1,0.008634,0.014236,0.008887,0.016074,0.018967,-0.005374,0.008907,0.005150,...,-0.005314,-0.005453,0.231470,81.134501,95.248063,101.018966,87.481590,104.334114,0.002927,0.013292
200,2013-10-17,1,-0.005374,0.008907,0.005150,0.018112,0.383985,0.016889,0.000334,0.012349,...,0.009986,0.002629,-0.392970,80.937425,94.963980,100.565934,87.481590,104.138252,0.002780,0.013345


## Merging Back Original Data

In [807]:
merged_df = df.merge(df_original, on='Date', how='inner')
merged_df


,Date,Close_change,log_Open_lag1,log_High_lag1,log_Low_lag1,log_Close_lag1,log_Volume_lag1,log_Open_lag2,log_High_lag2,log_Low_lag2,...,min,max,mom,vol,Open,High,Low,Close,Volume,Ticker
0,2024-03-20,1,-0.000832,0.003691,0.012428,0.007203,-0.216616,-0.004042,-0.016367,-0.018916,...,179.839996,227.839996,-0.003969,0.022578,178.990005,188.300003,178.600006,187.779999,12770000,BA
1,2024-03-19,1,-0.004042,-0.016367,-0.018916,-0.014847,-0.119987,-0.005565,-0.004118,0.002158,...,179.839996,229.000000,-0.006364,0.024518,180.110001,181.860001,179.740005,181.139999,7648000,BA
2,2024-03-18,0,-0.005565,-0.004118,0.002158,0.007589,0.066843,-0.006353,-0.005554,-0.008605,...,179.839996,249.000000,-0.006179,0.024657,180.259995,181.190002,177.520004,179.839996,9497800,BA
3,2024-03-15,1,-0.006353,-0.005554,-0.008605,-0.006603,-0.017802,-0.027358,-0.013937,-0.005914,...,181.149994,249.000000,-0.005798,0.024668,180.990005,184.179993,180.910004,182.529999,10708600,BA
4,2024-03-14,0,-0.027358,-0.013937,-0.005914,-0.010311,-0.607309,-0.031222,-0.038949,-0.035611,...,181.149994,249.000000,-0.006583,0.024857,182.000000,184.940002,180.520004,181.149994,10016200,BA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,2013-10-22,1,0.001550,-0.000977,-0.001568,-0.008607,-0.201546,0.022204,0.003833,0.011947,...,87.481590,104.334114,0.003225,0.013230,103.891324,105.424149,103.789133,104.300079,4431200,BA
2620,2013-10-21,0,0.022204,0.003833,0.011947,0.001879,-0.047282,0.008634,0.014236,0.008887,...,87.481590,104.334114,0.002851,0.013348,104.478871,104.521445,103.090811,103.439957,3207900,BA
2621,2013-10-18,1,0.008634,0.014236,0.008887,0.016074,0.018967,-0.005374,0.008907,0.005150,...,87.481590,104.334114,0.002927,0.013292,104.317086,104.623651,103.252626,104.334114,3924200,BA
2622,2013-10-17,1,-0.005374,0.008907,0.005150,0.018112,0.383985,0.016889,0.000334,0.012349,...,87.481590,104.138252,0.002780,0.013345,102.026361,104.223408,102.026361,104.138252,4114200,BA


In [808]:
df.to_csv(f'temp/{TICKER}.csv', index=False)
merged_df.to_csv(f'temp/{TICKER}_original.csv', index=False)

# News

In [809]:
def query_news_data(news_topic):
    try:
        # Connect to SQLite database
        conn = sqlite3.connect(config.DATABASE)

        # Perform SQL query
        query = f"""
            SELECT 
                cast(date as TEXT) as Date,
                title as Title,
                top_image as Top_Image,
                videos as Videos,
                url as URL,
                short_description as Short_Description,
                text as Text,
                source as Source,
                sentiment as Sentiment,
                sentiment_score as Sentiment_Score
            FROM ai_safety_news 
            WHERE search_topic = '{news_topic}'
            ORDER BY date DESC
        """
        data = pd.read_sql(query, conn)
        conn.close()
        return data
    except Exception as e:
        return pd.DataFrame()

In [810]:
news = query_news_data(news_topic="Boeing")

In [811]:
sentiment_counts = news.groupby(['Date', 'Sentiment']).size().unstack(fill_value=0)
sentiment_counts.columns = ['Negative', 'Neutral', 'Positive']
sentiment_counts


,Negative,Neutral,Positive
Date,,,
2014-04-03,0,0,1
2014-04-07,0,0,2
2014-04-10,0,0,1
2014-04-14,0,0,1
2014-05-06,0,0,1
...,...,...,...
2024-03-12,3,0,0
2024-03-13,1,0,1
2024-03-14,0,0,1


In [812]:

sentiment_counts['pct_news_Negative'] = sentiment_counts['Negative'] / sentiment_counts.sum(axis=1)
sentiment_counts['sma_news_Negative'] = sentiment_counts['pct_news_Negative'].rolling(window=7).mean()

sentiment_counts

sentiment_counts['sma_news_Negative']= sentiment_counts['sma_news_Negative'].astype(float).round(3)

In [813]:
final_df = df.merge(sentiment_counts.astype(str), left_on=df['Date'].astype(str), right_on=sentiment_counts.index.astype(str), how='left')
final_df.drop(columns=['key_0'], inplace=True)
final_df


/var/folders/bn/gpsh1csx5cn3n2b260wncbvm0000gn/T/ipykernel_1745/86015126.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final_df = df.merge(sentiment_counts.astype(str), left_on=df['Date'].astype(str), right_on=sentiment_counts.index.astype(str), how='left')


,Date,Close_change,log_Open_lag1,log_High_lag1,log_Low_lag1,log_Close_lag1,log_Volume_lag1,log_Open_lag2,log_High_lag2,log_Low_lag2,...,sma_short,min,max,mom,vol,Negative,Neutral,Positive,pct_news_Negative,sma_news_Negative
0,2024-03-20,1,-0.000832,0.003691,0.012428,0.007203,-0.216616,-0.004042,-0.016367,-0.018916,...,187.304001,179.839996,227.839996,-0.003969,0.022578,NaN,NaN,NaN,NaN,NaN
1,2024-03-19,1,-0.004042,-0.016367,-0.018916,-0.014847,-0.119987,-0.005565,-0.004118,0.002158,...,188.626001,179.839996,229.000000,-0.006364,0.024518,NaN,NaN,NaN,NaN,NaN
2,2024-03-18,0,-0.005565,-0.004118,0.002158,0.007589,0.066843,-0.006353,-0.005554,-0.008605,...,190.626001,179.839996,249.000000,-0.006179,0.024657,NaN,NaN,NaN,NaN,NaN
3,2024-03-15,1,-0.006353,-0.005554,-0.008605,-0.006603,-0.017802,-0.027358,-0.013937,-0.005914,...,192.696001,181.149994,249.000000,-0.005798,0.024668,1,0,1,0.5,0.524
4,2024-03-14,0,-0.027358,-0.013937,-0.005914,-0.010311,-0.607309,-0.031222,-0.038949,-0.035611,...,194.443001,181.149994,249.000000,-0.006583,0.024857,0,0,1,0.0,0.595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,2013-10-22,1,0.001550,-0.000977,-0.001568,-0.008607,-0.201546,0.022204,0.003833,0.011947,...,102.025521,87.481590,104.334114,0.003225,0.013230,NaN,NaN,NaN,NaN,NaN
2620,2013-10-21,0,0.022204,0.003833,0.011947,0.001879,-0.047282,0.008634,0.014236,0.008887,...,101.426015,87.481590,104.334114,0.002851,0.013348,NaN,NaN,NaN,NaN,NaN
2621,2013-10-18,1,0.008634,0.014236,0.008887,0.016074,0.018967,-0.005374,0.008907,0.005150,...,101.018966,87.481590,104.334114,0.002927,0.013292,NaN,NaN,NaN,NaN,NaN
2622,2013-10-17,1,-0.005374,0.008907,0.005150,0.018112,0.383985,0.016889,0.000334,0.012349,...,100.565934,87.481590,104.138252,0.002780,0.013345,NaN,NaN,NaN,NaN,NaN


In [814]:
!sgpt "summarize a dataframes numeric columns"


To summarize a dataframe's numeric columns, you can use the describe() function in Python. This will provide statistics such as count, mean, standard deviation, minimum, maximum, and quartiles for each numeric column in the dataframe.


In [815]:
final_df['sma_news_Negative'].astype(float).describe()


count    345.000000
mean       0.287780
std        0.199218
min        0.000000
25%        0.143000
50%        0.286000
75%        0.429000
max        0.810000
Name: sma_news_Negative, dtype: float64

In [816]:
final_df.to_csv(f'temp/{TICKER}_enriched.csv', index=False)